# FAT - Projet Vélib <a name="top"></a>


# Table of contents
1. [Simulation](#introduction)
2. [Calcul théorique](#theorie)

### Simulation <a name="introduction"></a> [$\uparrow$](#top)

Dans l'ensemble de cette section les matrices M de taille $NxN$ sont définies comme suit :
* sur la diagonale, les valeurs associées aux stations ($M[i][i]$ correspond à la station $i$)
* ailleurs, les valeurs associées aux transitions ($M[i][j]$ correspond à la transition $t_{ij}$, de $i$ vers $j$)

In [31]:
import random
import copy
import math

In [9]:
# Paramètres

N = 5

horizon = 150 * 60 * 60

tau = [[0,3,5,7,7],
      [2,0,2,5,5],
      [4,2,0,3,3],
      [8,6,4,0,2],
      [7,7,5,2,0]]

lambd_i = [2.8,3.7,5.5,3.5,4.6]

lambd = []
for i in range(N):
    lambd.append([])
    for j in range(N):
        if i==j:
            lambd[i].append(lambd_i[i] / 3600)
        else:
            lambd[i].append(1/(60*tau[i][j]))

p = [[0,0.22,0.32,0.2,0.26],
      [0.17,0,0.34,0.21,0.28],
      [0.19,0.26,0,0.24,0.31],
      [0.17,0.22,0.33,0,0.28],
      [0.18,0.24,0.35,0.23,0]]


In [10]:
# Lance une simulation avec les conditions initiales définies dans la fonction init() sur une période de temps "horizon"
def simulation(n_init):
    n = copy.deepcopy(n_init)
    temps = 0
    temps_vide = [0 for _ in range(N)]
    temps_vide2 = [[0 for _ in range(N)] for _ in range(N)]
    while temps < horizon:
        sum_q = 0
        for i in range(N):
            for j in range(N):
                if i == j:
                    if n[i][i] > 0:
                        sum_q += lambd[i][i]
                else:
                    sum_q += lambd[i][j] * n[i][j]
        tirage = random.expovariate(sum_q)
        temps += tirage
        # Calcul des temps vides
#        for i in range(N):
#            if(n[i][i] == 0):
#                temps_vide[i] += tirage
        for i in range(N):
            for j in range(N):
                if(n[i][j] == 0):
                    temps_vide2[i][j] += tirage

        #
        
        
        rand = random.random() * sum_q
        s = 0
        i = 0
        j = 0
        while 1 :
            if i == j:
                if n[i][i] > 0:
                    s += lambd[i][i]
            else:
                s += lambd[i][j] * n[i][j]
            if s >= rand :
                break
            if i == N-1:
                i = 0
                j += 1
            else:
                i += 1

        if i == j: # Départ de i
            s -= lambd[i][i]
            # On détermine la station destination
            j = 0
            while 1 :
                s += lambd[i][i] * p[i][j]
                if s >= rand:
                    break
                j += 1
            # On part de i vers j
            n[i][i] -= 1
            n[i][j] += 1
        else: # Arrivée en j à partir de i
            n[j][j] += 1
            n[i][j] -= 1
    return temps_vide2, n

In [24]:
def start_simulation(K, n_init):
    empty_time = [[0 for _ in range(N)] for _ in range(N)]
    empty_end = [[0 for _ in range(N)] for _ in range(N)]
    for _ in range(K):
        res, n_end = simulation(n_init)
        empty_time = [[empty_time[i][j] + res[i][j] for j in range(N)] for i in range(N)]
        empty_end = [[empty_end[i][j] + (1 if n_end[i][j] == 0 else 0) for j in range(N)] for i in range(N)] 
    empty_time = [[empty_time[i][j]/(K*horizon) for j in range(N)] for i in range(N)]
    empty_end = [[empty_end[i][j]/K for j in range(N)] for i in range(N)]
    return empty_time, empty_end

In [38]:
# Conditions initiales
n_init = [[20,1,0,0,0],
          [1,15,1,0,0],
          [0,1,17,1,0],
          [0,0,1,13,1],
          [0,0,0,1,18]]

K = 1000

empty_time, empty_end = start_simulation(K,n_init)

var = [(empty_end[i][i] - empty_end[i][i]**2)*K/(K-1) for i in range(N)]
epsilon = [1.96 * var[i] / math.sqrt(K) for i in range(N)]

In [39]:
print([empty_end[i][i] for i in range(N)])
print(var)
print(epsilon)
print([empty_end[i][i] - epsilon[i] for i in range(N)])
print([empty_end[i][i] + epsilon[i] for i in range(N)])


[0.009, 0.034, 0.145, 0.044, 0.09]
[0.008927927927927928, 0.03287687687687688, 0.1240990990990991, 0.04210610610610611, 0.08198198198198199]
[0.0005533587059463651, 0.0020377299403635412, 0.0076917418510708165, 0.002609763494441967, 0.005081295887095785]
[0.008446641294053633, 0.03196227005963646, 0.13730825814892916, 0.04139023650555803, 0.08491870411290421]
[0.009553358705946365, 0.03603772994036354, 0.15269174185107082, 0.046609763494441965, 0.09508129588709578]


In [ ]:
[[100-(100*m/horizon) for m in m2] for m2 in mean]

In [ ]:
[100-(100*mean[i][i]/horizon) for i in range(N)]

In [ ]:
[100-(100*mean[i][j]/horizon) for i in range(N) for j in range(N)]

In [ ]:
titi = [100-(100*mean[i][i]/horizon) for i in range(N)]
for i in range(N):
    for j in range(N):
        if(i!=j):
            titi.append(100-(100*mean[i][j]/horizon))
titi

In [ ]:
100 - sum(titi[0:5])

### Calcul théorique <a name="theorie"></a> [$\uparrow$](#top)